# Метод проекции градиента Розена

## Объяснение

In [106]:
import sympy as sym
import numpy as np

# переменные
x1, x2 = sym.symbols('x1 x2')

# целевая функция
f = x1**2 + x2**2

# ограничения
g = x1 + x2 - 1

# коэффициенты ограничений (без свободного члена b)
A = np.array([[1, 1]])

# начальная точка, удовлетворяющая ограничению
x0 = np.array([0, 1])

# переменные
variables = [x1,x2]

display(f, g)

x1**2 + x2**2

x1 + x2 - 1

In [107]:
# функция поиска обратной матрицы
inv = np.linalg.inv

# создаём единичную матрицу размерности n*n, где n - кол-во переменных
I = np.eye(len(variables))

# по формуле вычисляем матрицу P
P = I - A.T @ inv( A @ A.T ) @ A

P

array([[ 0.5, -0.5],
       [-0.5,  0.5]])

In [108]:
def get_gradient_at_point(variables: list[sym.Symbol], f: sym.Add, x0: np.ndarray):
    # градиент функции по всем переменным
    gradients_f = [sym.diff(f, variable) for variable in variables]
    
    # превращение из символьного представления в функцию
    lambd_gradients_f = [
        sym.lambdify(variables, gradient_f) for gradient_f in gradients_f]
    
    # вычисление градиента в точке x
    grad_f_x = np.array([f(*x0) for f in lambd_gradients_f])

    return np.where(np.abs(grad_f_x) > 0.0001, grad_f_x, 0)

In [109]:
# выясняем градиент функции в точке x0
grad_f_in_x = get_gradient_at_point(variables, f, x0)
grad_f_in_x

array([0, 2])

In [110]:
# по формуле находим направление S
S = -P @ grad_f_in_x 
S

array([ 1., -1.])

In [111]:
# переменная одномерной минимизации
λ = sym.symbols('λ')

# по формуле находим новый вектор, который зависит от одной переменной
new_vector = x0 + λ * S
new_vector

array([1.0*λ, 1 - 1.0*λ], dtype=object)

In [112]:
# заменяем каждую переменную функции на новую от λ (в данном случае [x1, -λ], [x2, λ+1] )
new_f = f.subs( list(zip(variables,new_vector)) )
new_f

1.0*λ**2 + (1 - 1.0*λ)**2

In [113]:
import pandas as pd
from typing import Callable

def fibonacci(
    f: Callable[[float], float],
    interval: tuple[float, float],
    eps: float,
    return_result_and_table: bool = False
) -> float:
    a, b = interval

    table = pd.DataFrame(
        columns=['a','b','l','x1','x2','f_x1','f_x2']
        )
    
    F = [1, 1]
    while F[-1] < (b - a) / eps:
        F.append(F[-1] + F[-2])
    
    delta = (b - a) / F[-1]
    
    N = len(F) - 1
    
    x1 = a + F[N - 2] * delta
    x2 = b - F[N - 2] * delta

    f_x1, f_x2 = f(x1), f(x2)

    table.loc[len(table)] = [a, b, b-a, x1, x2, f_x1, f_x2]

    for k in range(1, N - 2):
        if f_x1 < f_x2:
            b = x2
            x2, f_x2 = x1, f_x1
            x1 = a + F[N - k - 2] * delta
            f_x1 = f(x1)
        else:
            a = x1
            x1, f_x1 = x2, f_x2
            x2 = b - F[N - k - 2] * delta
            f_x2 = f(x2)
        table.loc[len(table)] = [a, b, b-a, x1, x2, f_x1, f_x2]
    
    table.loc[len(table)] = [a, b, b-a] + [None]*4

    if return_result_and_table:
        return (a + b) / 2, table
    return (a + b) / 2

In [114]:
# решаем задачу одномерной минимизации при помощи метода Фибоначчи
min_λ = fibonacci(f=sym.lambdify(λ, new_f), interval=[-5,5], eps=0.01)
min_λ

0.5009392611145899

In [115]:
# заменяем λ на минимальное значение, которое нашли при одномерной минимизации
[coordinate.subs(λ, min_λ) for coordinate in new_vector]

[0.500939261114590, 0.499060738885410]

## Градиент Розена

In [116]:
import sympy as sym
import numpy as np
import pandas as pd
from typing import Callable

def get_gradient_at_point(variables: list[sym.Symbol], f: sym.Add, x0: np.ndarray):
    # градиент функции по всем переменным
    gradients_f = [sym.diff(f, variable) for variable in variables]
    
    # превращение из символьного представления в функцию
    lambd_gradients_f = [
        sym.lambdify(variables, gradient_f) for gradient_f in gradients_f]
    
    # вычисление градиента в точке x
    grad_f_x = np.array([f(*x0) for f in lambd_gradients_f])

    return np.where(np.abs(grad_f_x) > 0.0001, grad_f_x, 0)

def fibonacci(
    f: Callable[[float], float],
    interval: tuple[float, float],
    eps: float,
    return_result_and_table: bool = False
) -> float:
    a, b = interval

    table = pd.DataFrame(
        columns=['a','b','l','x1','x2','f_x1','f_x2']
        )
    
    F = [1, 1]
    while F[-1] < (b - a) / eps:
        F.append(F[-1] + F[-2])
    
    delta = (b - a) / F[-1]
    
    N = len(F) - 1
    
    x1 = a + F[N - 2] * delta
    x2 = b - F[N - 2] * delta

    f_x1, f_x2 = f(x1), f(x2)

    table.loc[len(table)] = [a, b, b-a, x1, x2, f_x1, f_x2]

    for k in range(1, N - 2):
        if f_x1 < f_x2:
            b = x2
            x2, f_x2 = x1, f_x1
            x1 = a + F[N - k - 2] * delta
            f_x1 = f(x1)
        else:
            a = x1
            x1, f_x1 = x2, f_x2
            x2 = b - F[N - k - 2] * delta
            f_x2 = f(x2)
        table.loc[len(table)] = [a, b, b-a, x1, x2, f_x1, f_x2]
    
    table.loc[len(table)] = [a, b, b-a] + [None]*4

    if return_result_and_table:
        return (a + b) / 2, table
    return (a + b) / 2

def rosen_gradient(
    variables: list[sym.Symbol],
    f: sym.Add,
    A: np.ndarray,
    x0: np.ndarray,
    interval_for_one_dimensional_minimiz: list[float],
    eps: float
):
    # функция поиска обратной матрицы
    inv = np.linalg.inv

    # создаём единичную матрицу размерности n*n, где n - кол-во переменных
    I = np.eye(len(variables))

    # по формуле вычисляем матрицу P
    P = I - A.T @ inv( A @ A.T ) @ A

    # выясняем градиент функции в точке x0
    grad_f_in_x = get_gradient_at_point(variables, f, x0)

    # по формуле находим направление S
    S = -P @ grad_f_in_x

    # переменная одномерной минимизации
    λ = sym.symbols('λ')

    # по формуле находим новый вектор, который зависит от одной переменной
    new_vector = x0 + λ * S

    # заменяем каждую переменную функции на новую от λ
    new_f = f.subs( list(zip(variables, new_vector)) )

    # решаем задачу одномерной минимизации при помощи метода Фибоначчи
    min_λ = fibonacci(f=sym.lambdify(λ, new_f), interval=interval_for_one_dimensional_minimiz, eps=eps)

    # заменяем λ на минимальное значение, которое нашли при одномерной минимизации
    return [coordinate.subs(λ, min_λ) for coordinate in new_vector]

## Проверка

In [117]:
# переменные
x1, x2 = sym.symbols('x1 x2')

# целевая функция
f = x1**2 + x2**2

# коэффициенты ограничений (без свободного члена b)
A = np.array([[1, 1]])

# начальная точка, удовлетворяющая ограничению
x0 = np.array([0, 1])

# переменные
variables = [x1,x2]

rosen_gradient(variables, f, A, x0, interval_for_one_dimensional_minimiz=[-100, 100], eps=0.01)

[0.502495027392954, 0.497504972607046]